In [ ]:
!pip install pydub

In [24]:
def int_to_fa(num):
    d = { 0 : 'صِفر', 1 : '1ک', 2 : 'دُ', 3 : 'سه', 4 : 'چاهار', 5 : 'پَنج',
          6 : 'شِش', 7 : 'هَفت', 8 : 'هَشت', 9 : 'نُه', 10 : 'دَه',
          11 : '1ازدَه', 12 : 'دَوازدَه', 13 : 'سیزدَه', 14 : 'چاهاردَه',
          15 : 'پانزدَه', 16 : 'شانزدَه', 17 : 'هِفدَه', 18 : 'هِجدَه',
          19 : 'نوزدَه', 20 : 'بیست',
          30 : 'سی', 40 : 'چِهل', 50 : 'پَنجاه', 60 : 'شَصت',
          70 : 'هَفتاد', 80 : 'هَشتاد', 90 : 'نَوَد' }
    k = 1000
    m = k * 1000
    b = m * 1000
    t = b * 1000

    assert(0 <= num)

    if (num < 20):
        return d[num]

    if (num < 100):
        if num % 10 == 0: return d[num]
        else: return d[num // 10 * 10] + ' و ' + d[num % 10]

    if (num < k):
      if (200 <= num <300):
          if num % 100 == 0: return d[num // 100] + ' دیویست'
          else: return  ' دیویست وَ ' + int_to_fa(num % 100)
      elif (300 <= num <400):
          if num % 100 == 0: return d[num // 100] + ' سیصَد'
          else: return  ' سیصَد وَ ' + int_to_fa(num % 100)
      elif (400 <= num <500):
          if num % 100 == 0: return d[num // 100] + ' چهارصَد'
          else: return  ' چهارصَد وَ ' + int_to_fa(num % 100)
      elif (500 <= num <600):
          if num % 100 == 0: return d[num // 100] + ' پانصَد'
          else: return ' پانصَد وَ ' + int_to_fa(num % 100)
      else:
          if num % 100 == 0: return d[num // 100] + ' صَد'
          else: return d[num // 100] + ' صَد وَ ' + int_to_fa(num % 100)

    if (num < m):
        if num % k == 0: return int_to_fa(num // k) + ' هِزار'
        else: return int_to_fa(num // k) + ' هِزار وَ ' + int_to_fa(num % k)

    if (num < b):
        if (num % m) == 0: return int_to_fa(num // m) + ' میل1ؤن'
        else: return int_to_fa(num // m) + ' میل1ؤن وَ ' + int_to_fa(num % m)

    if (num < t):
        if (num % b) == 0: return int_to_fa(num // b) + ' میل1ارد'
        else: return int_to_fa(num // b) + ' میل1ارد وَ ' + int_to_fa(num % b)

    if (num % t == 0): return int_to_fa(num // t) + ' تریل1ؤن'
    else: return int_to_fa(num // t) + ' تریل1ؤن وَ ' + int_to_fa(num % t)

    raise AssertionError('عدد بیش از اندازه بزرگ است: %s' % str(num))


In [37]:
from pydub import AudioSegment
import os
def detect_leading_silence(sound, silence_threshold=-90.0, chunk_size=10):
    '''
    sound: a pydub.AudioSegment
    silence_threshold: in dB
    chunk_size: in ms
    iterate over chunks until you find the first one with sound
    '''
    trim_ms = 0  # ms
    assert chunk_size > 0  # to avoid infinite loop
    while sound[trim_ms:trim_ms + chunk_size].dBFS < silence_threshold and trim_ms < len(sound):
        trim_ms += chunk_size
    return trim_ms

def concatenate_phoneme_audio(text_input, phoneme_mapping, fade_duration=90, overlap_duration=1500):
    output_audio = AudioSegment.silent()

    for i, char in enumerate(text_input):
        phoneme = phoneme_mapping.get(char)
        if phoneme:
            audio_file_path = f"Phoneme/{phoneme}.wav"  # Replace with the actual path to your audio files
            if os.path.exists(audio_file_path):
                phoneme_audio = AudioSegment.from_wav(audio_file_path)
                # start_trim = detect_leading_silence(phoneme_audio)
                # end_trim = detect_leading_silence(phoneme_audio.reverse())

                # # Trim the audio
                # duration = len(phoneme_audio)
                # phoneme_audio = phoneme_audio[start_trim:duration - end_trim]
                duration = len(phoneme_audio)
                #phoneme_audio=phoneme_audio[:int(duration/1.3)]
                # Apply fade-in and fade-out to each phoneme audio segment
                phoneme_audio = (
                    phoneme_audio
                    .fade_in(fade_duration)
                    .fade_out(fade_duration)
                    .apply_gain_stereo(0.8)  # Adjust the stereo balance if needed
                )

                output_audio += phoneme_audio
            else:
                print(f"Audio file not found for phoneme: {phoneme}")

    return output_audio


persian_phoneme_mapping = {
    'ی': 'I',
    'ِ': 'e',
    'َ': 'a',
    'ؤ': 'u',
    'ُ': 'o',
    'آ': 'AA',
    'ا': 'AA',
    'پ': 'p',
    'ب': 'b',
    'ت': 't',
    'ط': 't',
    'د': 'd',
    'ک': 'k',
    'گ': 'g',
    'ق': 'q',
    'ع': 'Eh',
    'ء': 'Eh',
    'أ': 'Eh',
    'چ': 'ch',
    'ج': 'j',
    'ف': 'f',
    'و': 'v',
    'س': 's',
    'ص': 's',
    'ث': 's',
    'ز': 'z',
    'ظ': 'z',
    'ض': 'z',
    'ذ': 'z',
    'ش': 'sh',
    'ژ': 'zh',
    'خ': 'kh',
    'ه': 'h',
    'ح': 'h',
    'ل': 'l',
    'ر': 'r',
    'م': 'm',
    'ن': 'n',
    '1': 'y',
}



<_io.BufferedRandom name='0000.mp3'>

In [43]:
input_str="دانِشگاهِ تِهران"

if input_str.isdigit():
    input_str = int_to_fa(int(input_str))

output_audio = concatenate_phoneme_audio(input_str, persian_phoneme_mapping)
output_audio.export(f"{input_str}.wav", format="wav")

<_io.BufferedRandom name='0000.wav'>

In [44]:
input_str="بِه کَجا چِنین شِتابان"

if input_str.isdigit():
    input_str = int_to_fa(int(input_str))

output_audio = concatenate_phoneme_audio(input_str, persian_phoneme_mapping)
output_audio.export(f"{input_str}.wav", format="wav")

<_io.BufferedRandom name='0000.wav'>

In [45]:
input_str="مُحَممَد جَوادِ رَنجبَر"

if input_str.isdigit():
    input_str = int_to_fa(int(input_str))

output_audio = concatenate_phoneme_audio(input_str, persian_phoneme_mapping)
output_audio.export(f"{input_str}.wav", format="wav")

<_io.BufferedRandom name='0000.wav'>

In [46]:
input_str="1409"

if input_str.isdigit():
    input_str = int_to_fa(int(input_str))

output_audio = concatenate_phoneme_audio(input_str, persian_phoneme_mapping)
output_audio.export(f"{input_str}.wav", format="wav")

<_io.BufferedRandom name='0000.wav'>

In [47]:
input_str="دِویست وَ دُ هِزار وَ دَه"

if input_str.isdigit():
    input_str = int_to_fa(int(input_str))

output_audio = concatenate_phoneme_audio(input_str, persian_phoneme_mapping)
output_audio.export(f"{input_str}.wav", format="wav")

<_io.BufferedRandom name='0000.wav'>